In [2]:
!pip3 install torch 
!pip3 install torchvision
!pip3 install tqdm

    100% |████████████████████████████████| 519.5MB 35kB/s 
tcmalloc: large alloc 1073750016 bytes == 0x5956a000 @  0x7fc2843022a4 0x594e17 0x626104 0x51190a 0x4f5277 0x510c78 0x5119bd 0x4f5277 0x4f3338 0x510fb0 0x5119bd 0x4f5277 0x4f3338 0x510fb0 0x5119bd 0x4f5277 0x4f3338 0x510fb0 0x5119bd 0x4f6070 0x510c78 0x5119bd 0x4f5277 0x4f3338 0x510fb0 0x5119bd 0x4f6070 0x4f3338 0x510fb0 0x5119bd 0x4f6070
    100% |████████████████████████████████| 61kB 4.9MB/s 
    100% |████████████████████████████████| 2.0MB 10.9MB/s 
  Found existing installation: Pillow 4.0.0
    Uninstalling Pillow-4.0.0:
      Successfully uninstalled Pillow-4.0.0


In [0]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [4]:
! wget -O ./text_files.tar.gz 'https://piazza.com/redirect/s3?bucket=uploads&prefix=attach%2Fjlifkda6h0x5bk%2Fhzosotq4zil49m%2Fjn13x09arfeb%2Ftext_files.tar.gz' 
! tar -xzf text_files.tar.gz
! pip install unidecode
! pip install torch

--2018-10-13 05:42:26--  https://piazza.com/redirect/s3?bucket=uploads&prefix=attach%2Fjlifkda6h0x5bk%2Fhzosotq4zil49m%2Fjn13x09arfeb%2Ftext_files.tar.gz
Resolving piazza.com (piazza.com)... 52.20.136.189, 52.55.184.84, 34.200.202.18, ...
Connecting to piazza.com (piazza.com)|52.20.136.189|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://d1b10bmlvqabco.cloudfront.net/attach/jlifkda6h0x5bk/hzosotq4zil49m/jn13x09arfeb/text_files.tar.gz [following]
--2018-10-13 05:42:26--  https://d1b10bmlvqabco.cloudfront.net/attach/jlifkda6h0x5bk/hzosotq4zil49m/jn13x09arfeb/text_files.tar.gz
Resolving d1b10bmlvqabco.cloudfront.net (d1b10bmlvqabco.cloudfront.net)... 143.204.98.78, 143.204.98.189, 143.204.98.56, ...
Connecting to d1b10bmlvqabco.cloudfront.net (d1b10bmlvqabco.cloudfront.net)|143.204.98.78|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1533290 (1.5M) [application/x-gzip]
Saving to: ‘./text_files.tar.gz’

./text_files.tar.gz 10

In [32]:
import unidecode
import string
import random
import re
 
import pdb
 
all_characters = string.printable # used for one hot encoding
n_characters = len(all_characters)
 
file1 = unidecode.unidecode(open('./text_files/lotr.txt').read())
file2 = unidecode.unidecode(open('./text_files/alma.txt').read())
file3 = unidecode.unidecode(open('./all_tswift_lyrics.txt').read()) # taylor swift lyrics
file4 = unidecode.unidecode(open('./sense_and_sensibility.txt').read()) # first 5 chapters of sense and sensibility
file1_len = len(file1)
print('file_len =', file_len)

file_len = 2579888


In [40]:
chunk_len = 200 # chunks are like batches
 
def random_chunk(file, file_len):
    start_index = random.randint(0, file_len - chunk_len)
    end_index = start_index + chunk_len + 1
    return file[start_index:end_index]
 
print(random_chunk(file1, file1_len))

The three others were far away in the North. In 
the house of Elrond it is told that they were at Ann(r)minas, and Amon Syl, 
and Elendil's Stone was on the Tower Hills that look towards Mithlond in th


In [38]:
import torch
from torch.autograd import Variable
# Turn string into list of longs
def char_tensor(string):
    tensor = torch.zeros(len(string)).long()
    for c in range(len(string)):
        tensor[c] = all_characters.index(string[c])
    return Variable(tensor) # tells you location of character in all_characters
  
print(char_tensor('abcDEF'))

def random_training_set(file):    
    chunk = random_chunk(file, len(file))
    inp = char_tensor(chunk[:-1])
    target = char_tensor(chunk[1:])
    return inp, target

tensor([10, 11, 12, 39, 40, 41])


# RNN and GRU

In [0]:
import torch
import torch.nn as nn
from torch.autograd import Variable
 
class GRU(nn.Module):
  def __init__(self, input_size, hidden_size, n_layers):
    super(GRU, self).__init__()
    self.input_size = input_size
    self.hidden_size = hidden_size
    self.n_layers = n_layers
    
    self.rt = nn.Sequential(nn.Linear(self.input_size + self.hidden_size,  self.hidden_size, bias=True), nn.Sigmoid())
    self.zt = nn.Sequential(nn.Linear(self.input_size + self.hidden_size, self.hidden_size, bias=True), nn.Sigmoid())
    self.nt = nn.Sequential(nn.Linear(self.input_size + self.hidden_size, self.hidden_size, bias=True), nn.Tanh())
  
  def forward(self, input, hidden):
    # concatenates input and hidden layer 
    temp = torch.cat((input, hidden), dim=2)
    r = self.rt(temp)
    z = self.zt(temp)
    
    # concatenates input and r*hidden layer 
    rh = torch.cat((input, r*hidden), dim=2)
    n = self.nt(rh)
    h = (1-z)*n +z*hidden
    
    return h, h
    
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, n_layers=1):
        super(RNN, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.n_layers = n_layers
 
        # encode using embedding layer
        self.embedding = nn.Embedding(input_size, hidden_size)
    
        # set up GRU passing in number of layers parameter (nn.GRU)
        self.gru = GRU(hidden_size, hidden_size, n_layers)
      
        # decode output
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)
 
    def forward(self, input, hidden):
        # by reviewing the documentation, construct a forward function that properly uses the output
        # of the GRU
        embedded = self.embedding(input).view(1, 1, -1)
        output = F.relu(embedded)
        output, hidden = self.gru(output, hidden)
        output = self.softmax(self.out(output[0]))
        
        # return output and hidden
        return output, hidden
 
    def init_hidden(self):
        return Variable(torch.zeros(self.n_layers, 1, self.hidden_size))


# Train and Predict

In [0]:
def train(inp, target):
    ## initialize hidden layers, set up gradient and loss 
    hidden = decoder.init_hidden()
    decoder_optimizer.zero_grad()

    loss = 0
    for c in range(chunk_len):
        output, hidden = decoder(inp[c], hidden) # run the forward pass of your rnn with proper input
        loss += criterion(output, target[c].unsqueeze(0))
 
    ## calculate backwards loss and step the optimizer (globally)
    loss.backward()
    decoder_optimizer.step()

 
    return loss.item() / chunk_len

def evaluate(prime_str='A', predict_len=100, temperature=0.8):
    ## initialize hidden variable, initialize other useful variables 
    hidden = decoder.init_hidden()
    prime_input = char_tensor(prime_str)
    predicted = prime_str
 
    # Use priming string to "build up" hidden state
    for p in range(len(prime_str) - 1):
        _, hidden = decoder(prime_input[p], hidden)
    inp = prime_input[-1]
 
    for p in range(predict_len):
        output, hidden = decoder(inp, hidden) # run your RNN/decoder forward on the input
 
        # Sample from the network as a multinomial distribution
        output_dist = output.data.view(-1).div(temperature).exp()
        top_i = torch.multinomial(output_dist, 1)[0]
 
        ## get character from your list of all characters, add it to your output str sequence, set input
        ## for the next pass through the model
    
        predicted += all_characters[top_i.item()]
        inp = top_i      

    return predicted

# Lord of the Rings

In [25]:
import time
n_epochs = 2000
print_every = 100
plot_every = 10
hidden_size = 100
n_layers = 1
lr = 0.005
 
decoder = RNN(n_characters, hidden_size, n_characters, n_layers)
decoder_optimizer = torch.optim.Adam(decoder.parameters(), lr=lr)
criterion = nn.CrossEntropyLoss()
 
start = time.time()
all_losses = []
loss_avg = 0
 
for epoch in range(1, n_epochs + 1):
    loss_ = train(*random_training_set(file1))       
    loss_avg += loss_
 
    if epoch % print_every == 0:
        print('[%s (%d %d%%) %.4f]' % (time.time() - start, epoch, epoch / n_epochs * 100, loss_))
        print(evaluate('Wh', 100), '\n')
 
    if epoch % plot_every == 0:
        all_losses.append(loss_avg / plot_every)
        loss_avg = 0

[20.1297664642334 (100 5%) 2.4010]
Whand saand 


thand Tto smos raf ing. wererer ouh 



ok me ruthe renoned. fe storeis end and wi ente 

[40.125828981399536 (200 10%) 2.0951]
Wh themily sall mater. 'Bo forrmorrted muclinddaved and the cam shall crathe andonln be oust frondi'n  

[60.323487281799316 (300 15%) 1.9893]
Whe fat tro Hrode gromtalpen And went, sowen 
waco rim not bampit frorgo wean tubut wile, wive not ot  

[80.403480052948 (400 20%) 2.1765]
Wh gast Last of take wed, and and to with me trey and frame a were ney the 
frean. 


There of anr you 

[100.50981187820435 (500 25%) 1.8353]
Wh with is ner 

his in 

the biedorn and the he sleet, Smaws of hill tien will in thingth!' The clowl 

[120.60929751396179 (600 30%) 1.9324]
Wh ligh his we hither thoued on the stroud see? Frod of the said they in befut if the caming and bene  

[140.72434329986572 (700 35%) 1.6532]
Whers in 
the doows of thell so of the 
call may not had must on the pid det. Tink, not to a 
leed. Bu 

[160.

# Alma

In [41]:
decoder = RNN(n_characters, hidden_size, n_characters, n_layers)
decoder_optimizer = torch.optim.Adam(decoder.parameters(), lr=lr)
criterion = nn.CrossEntropyLoss()
 
start = time.time()
all_losses = []
loss_avg = 0
 
for epoch in range(1, n_epochs + 1):
    loss_ = train(*random_training_set(file2))       
    loss_avg += loss_
 
    if epoch % print_every == 0:
        print('[%s (%d %d%%) %.4f]' % (time.time() - start, epoch, epoch / n_epochs * 100, loss_))
        print(evaluate('Wh', 100), '\n')
 
    if epoch % plot_every == 0:
        all_losses.append(loss_avg / plot_every)
        loss_avg = 0

[20.150388956069946 (100 5%) 2.1082]
Whis and wore she sam to whe heured in deopt
 hes of the ano Lour hed Ae hauntis ien hevim we: toreche 

[39.99161720275879 (200 10%) 1.9285]
Wh yere hat been nind God of the Not rouf the purwe Jeord; Tere jeiul the seow kes on meal the God os  

[59.98768329620361 (300 15%) 1.7453]
Whesd or themy or eveais in whis farts to he sorthre mento behief thor ith loly unto sil thould and ou 

[79.78358769416809 (400 20%) 1.6565]
Whed, of the ling, the sand the hast agaid pak the nore Lead and I sanien liveren be came to saing int 

[99.60424184799194 (500 25%) 1.5468]
Whes are to san of terent abe has belth hear did grefwatiesers and man, and the copCed and did nom thu 

[119.55217242240906 (600 30%) 1.3159]
Whou? I said: And the srought thatious kestore, the Lamanites of a the areir he which hum.

 And of th 

[139.45719194412231 (700 35%) 1.5432]
Whe mecation, the Lord is the chich ye should nis cass; and hild vaccingech that we lignown, and he sh 

[159

# Taylor Swift lyrics

In [42]:
decoder = RNN(n_characters, hidden_size, n_characters, n_layers)
decoder_optimizer = torch.optim.Adam(decoder.parameters(), lr=lr)
criterion = nn.CrossEntropyLoss()
 
start = time.time()
all_losses = []
loss_avg = 0
 
for epoch in range(1, n_epochs + 1):
    loss_ = train(*random_training_set(file3))       
    loss_avg += loss_
 
    if epoch % print_every == 0:
        print('[%s (%d %d%%) %.4f]' % (time.time() - start, epoch, epoch / n_epochs * 100, loss_))
        print(evaluate('Wh', 100), '\n')
 
    if epoch % plot_every == 0:
        all_losses.append(loss_avg / plot_every)
        loss_avg = 0

[20.435955286026 (100 5%) 2.3350]
Whoor sow'ind yow wh oo somy kis whash,
Ang you lay kiper ho rinl gachare barit hathinl wou baster fiw 

[40.99764823913574 (200 10%) 1.9893]
Whe syou he the sae in the say's
You rere sthy, stayot to koung yust

But
Buthe nowe you like sta ghow 

[61.663105726242065 (300 15%) 2.1588]
Wher ong
That arat reand
Shour bere sorow, to a un tere sime
2n in prous sole, on't of and
And tere se 

[82.06050777435303 (400 20%) 1.8977]
Whing sher, sich shownt way to and Ip thired?
I waing this a for whond wails am sall dimed you'd eve p 

[102.67081117630005 (500 25%) 1.9534]
Whe cat whe do betrout where wer a well laick a got with anrerout you do back the way faldy some...... 

[123.15146708488464 (600 30%) 1.7862]
Whing we al in 
I night to feld enough your hor can night it coom
All you love it's gong
You in to you 

[143.59419107437134 (700 35%) 1.7957]
Wher am baby, I want it got fase
I (xchemm was of in was us flore your haget a rlay
I love you wanna I 

[163.

# Sense and Sensibility

In [43]:
decoder = RNN(n_characters, hidden_size, n_characters, n_layers)
decoder_optimizer = torch.optim.Adam(decoder.parameters(), lr=lr)
criterion = nn.CrossEntropyLoss()
 
start = time.time()
all_losses = []
loss_avg = 0

for epoch in range(1, n_epochs + 1):
    loss_ = train(*random_training_set(file4))       
    loss_avg += loss_
 
    if epoch % print_every == 0:
        print('[%s (%d %d%%) %.4f]' % (time.time() - start, epoch, epoch / n_epochs * 100, loss_))
        print(evaluate('Wh', 100), '\n')
 
    if epoch % plot_every == 0:
        all_losses.append(loss_avg / plot_every)
        loss_avg = 0

[20.71542763710022 (100 5%) 2.2608]
Wh thidilaug he. athirI be whe whe ave he house erte lobe th thiser7son evg mior whed das was "hordee  

[41.477357149124146 (200 10%) 2.0774]
Whe she in to foran has of her yond of hat hind hat co could tas and rish oup hat in an a was ath an b 

[62.25443077087402 (300 15%) 1.9960]
Wh. As she prou her all reashelind worice Ture. Nhenr was wuch in vere fnenfore wion in-lito her mand  

[83.31416916847229 (400 20%) 1.6512]
Wh the thougome od of halnaint for that his as leithald ir esparle and prom me the the persant he enou 

[103.97436022758484 (500 25%) 1.7920]
Whadune, the give shat of than, in unser where? he proponsr to pichwood Mothery in the at them, be may 

[124.57213068008423 (600 30%) 1.6749]
Wh, fortate was may, withen othee atget my her simably as Maris-in her mich to house's theart as in th 

[145.23896050453186 (700 35%) 1.7579]
Wh were of be with semely at ald and she mowifle of exceenession I deade her sel hat same the lome, hi 

[16